<a href="https://colab.research.google.com/github/triandosimarmata/myprojects/blob/main/LM_CMAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import cma
import numpy as np
import matplotlib.pyplot as plt

# 1. DEFINISI FUNGSI OBJEKTIF (ROTATED ELLIPSOID)
# Fungsi ini tetap sama, namun akan dievaluasi pada ruang berdimensi sangat tinggi.
class RotatedEllipsoid:
    def __init__(self, dimension, alpha=10):
        self.dimension = dimension
        self.alpha = alpha
        # Membuat matriks rotasi acak
        H = np.random.rand(self.dimension, self.dimension)
        self.rotation_matrix, _ = np.linalg.qr(H)
        # Membuat vektor bobot untuk sumbu elips
        self.weights = np.array([10**(self.alpha * i / (self.dimension - 1))
                                 for i in range(self.dimension)])

    def __call__(self, x):
        if len(x) != self.dimension:
            raise ValueError(f"Input vector must have dimension {self.dimension}")
        # Rotasi vektor input
        y = np.dot(self.rotation_matrix, x)
        # Menghitung nilai fungsi
        return np.sum((self.weights * y)**2)

# 2. SETUP DAN EKSEKUSI CMA-ES UNTUK MASALAH SKALA BESAR
if __name__ == '__main__':
    # Parameter untuk optimasi: Dimensi sangat tinggi
    DIMENSION = 100  # <--- Inilah domain untuk LM-CMA-ES
    STARTING_POINT = np.random.rand(DIMENSION) * 10 - 5
    INITIAL_STD_DEV = 0.5

    # Membuat instance dari fungsi rotated ellipsoid
    objective_function = RotatedEllipsoid(dimension=DIMENSION)

    # Inisialisasi algoritma CMA-ES.
    # Untuk N=500, pycma akan menangani tantangan matriks kovarians yang besar,
    # sejalan dengan prinsip di balik LM-CMA-ES.
    es = cma.CMAEvolutionStrategy(STARTING_POINT, INITIAL_STD_DEV)

    print(f"Memulai optimasi untuk fungsi Rotated Ellipsoid di {DIMENSION} dimensi.")
    print("Ini mungkin memakan waktu beberapa saat karena dimensinya sangat tinggi.")
    print("-" * 30)

    # Menjalankan loop optimasi
    # Perhatikan bagaimana setiap iterasi (generasi) akan lebih lambat daripada
    # contoh sebelumnya yang hanya 10 dimensi.
    while not es.stop():
        solutions = es.ask()
        fitness_values = [objective_function(x) for x in solutions]
        es.tell(solutions, fitness_values)
        es.disp()

    # 3. MENAMPILKAN HASIL AKHIR
    print("-" * 30)
    print("Optimasi Selesai!")

    best_solution = es.result.xbest
    best_fitness = es.result.fbest

    print(f"Nilai fungsi minimum (f(x)) yang ditemukan: {best_fitness:.6e}")
    # Menampilkan semua elemen dari vektor solusi terbaik
    print(f"Solusi terbaik: {np.round(best_solution, 4)}")

    # Plot hasil evolusi
    cma.plot()
    plt.show()

(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=769841, Wed Sep 24 10:56:16 2025)
Memulai optimasi untuk fungsi Rotated Ellipsoid di 100 dimensi.
Ini mungkin memakan waktu beberapa saat karena dimensinya sangat tinggi.
------------------------------
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 5.494324189742345e+20 1.0e+00 4.82e-01  5e-01  5e-01 0:00.0
    2     34 3.637701443167122e+20 1.0e+00 4.69e-01  5e-01  5e-01 0:00.0
    3     51 2.426063385510742e+20 1.0e+00 4.58e-01  5e-01  5e-01 0:00.0
  100   1700 2.657776204968188e+19 1.4e+00 3.69e-01  4e-01  4e-01 0:00.8
  200   3400 1.305192705088387e+19 1.7e+00 3.51e-01  3e-01  4e-01 0:01.6
  300   5100 7.255519035194845e+18 1.8e+00 1.98e-01  2e-01  2e-01 0:02.1
  400   6800 4.771178603632358e+18 2.0e+00 1.84e-01  2e-01  2e-01 0:02.5
  500   8500 4.245086373153335e+18 2.2e+00 1.47e-01  1e-01  2e-01 0:02.9
  600  10200 2.202923510020498e+18 2.5e+00 9.39e-02  9e-02  1e-01 0:03.2
  700  119

/usr/local/lib/python3.12/dist-packages/cma/utilities/utils.py:362: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.0e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (time=Sep 24 10:58:49 2025 class=CMAEvolutionStrategy method=alleviate_conditioning iteration=31501)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


31600 537200 1.917782828140982e+09 2.3e+00 2.16e-04  2e-04  3e-04 2:24.8
31700 538900 1.752745692466931e+09 4.6e+00 3.75e-04  4e-04  6e-04 2:25.2
31800 540600 1.546686768324265e+09 7.8e+00 4.45e-04  4e-04  1e-03 2:25.6
31900 542300 1.438931569166660e+09 1.0e+01 1.91e-04  2e-04  6e-04 2:26.0
32000 544000 1.428497761016599e+09 1.1e+01 6.09e-05  6e-05  2e-04 2:26.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
32100 545700 1.427608356779677e+09 1.1e+01 1.58e-05  1e-05  5e-05 2:26.8
32200 547400 1.427400260167644e+09 1.2e+01 1.26e-05  1e-05  4e-05 2:27.2
32300 549100 1.426855833176637e+09 1.2e+01 2.56e-05  2e-05  8e-05 2:27.6
32400 550800 1.424691002603582e+09 1.2e+01 5.16e-05  5e-05  2e-04 2:28.1
32500 552500 1.416446567333262e+09 1.8e+01 1.12e-04  1e-04  4e-04 2:28.5
32600 554200 1.376343446947757e+09 3.6e+01 2.04e-04  2e-04  1e-03 2:28.9
32700 555900 1.269426920211312e+09 6.8e+01 3.27e-04  3e-04  4e-03 2:29.3
32800 557600 1.040549887186688e+09 1.3e+02 4.54e-04 

/usr/local/lib/python3.12/dist-packages/cma/logger.py:378: UserWarning: ast.literal_eval("None") read from file /content/outcmaes/parameters.json2 failed with Exception malformed node or string: None
  warnings.warn('ast.literal_eval("{0}") read from file {1} failed with Exception {2}'.format(self.parameters, fn, e))
/usr/local/lib/python3.12/dist-packages/cma/logger.py:416: UserWarning: reading from file "/content/outcmaes/axlen.dat" failed (time=Sep 24 11:03:21 2025 class=CMADataLogger method=load)
  'load', 'CMADataLogger'); m and warnings.warn(m)
/usr/local/lib/python3.12/dist-packages/cma/logger.py:416: UserWarning: reading from file "/content/outcmaes/axlencorr.dat" failed (time=Sep 24 11:03:21 2025 class=CMADataLogger method=load)
  'load', 'CMADataLogger'); m and warnings.warn(m)
/usr/local/lib/python3.12/dist-packages/cma/logger.py:416: UserWarning: reading from file "/content/outcmaes/axlenprec.dat" failed (time=Sep 24 11:03:21 2025 class=CMADataLogger method=load)
  'load', 

In [2]:
%pip install cma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 6.9 MB/s eta 0:00:00
